# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,170.006645,0.384855,1234,170.391500,169.621790,340.783001,339.243580
6,75.351000,0.221204,1234,75.572204,75.129796,151.144409,150.259592
7,476.557653,0.216625,1234,476.774278,476.341028,953.548556,952.682057
8,677.298267,0.976453,1234,678.274720,676.321814,1356.549440,1352.643628
9,570.495055,-0.198326,1234,570.296728,570.693381,1140.593457,1141.386762
10,628.371899,0.429112,1234,628.801011,627.942788,1257.602022,1255.885575
11,51.747836,0.958435,1234,52.706271,50.789401,105.412541,101.578802
12,696.273928,-0.538880,1234,695.735047,696.812808,1391.470095,1393.625617
13,84.678198,-0.636077,1234,84.042121,85.314275,168.084242,170.628550
14,823.096922,-0.087414,1234,823.009508,823.184335,1646.019016,1646.368670


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-170.391500,169.621790
6,-75.572204,75.129796
7,-476.774278,476.341028
8,-678.274720,676.321814
9,-570.296728,570.693381
10,-628.801011,627.942788
11,-52.706271,50.789401
12,-695.735047,696.812808
13,-84.042121,85.314275
14,-823.009508,823.184335


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-170.391500,169.621790
6,-75.572204,75.129796
7,-476.774278,476.341028
8,-678.274720,676.321814
9,-570.296728,570.693381
10,-628.801011,627.942788
11,-52.706271,50.789401
12,-695.735047,696.812808
13,-84.042121,85.314275
14,-823.009508,823.184335


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-170.39150044,  169.62178978],
       [ -75.5722044 ,   75.12979612],
       [-476.77427807,  476.34102843],
       [-678.27471984,  676.32181406],
       [-570.29672837,  570.69338123],
       [-628.8010111 ,  627.94278761],
       [ -52.70627072,   50.78940096],
       [-695.73504741,  696.81280831],
       [ -84.04212111,   85.31427494],
       [-823.00950795,  823.18433511]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,338.783001,-339.243580
6,149.144409,-150.259592
7,951.548556,-952.682057
8,1354.549440,-1352.643628
9,1138.593457,-1141.386762
10,1255.602022,-1255.885575
11,103.412541,-101.578802
12,1389.470095,-1393.625617
13,166.084242,-170.628550
14,1644.019016,-1646.368670


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.170007,0.692428
6,0.075351,0.610602
7,0.476558,0.608312
8,0.677298,0.988226
9,0.570495,0.400837
10,0.628372,0.714556
11,0.051748,0.979217
12,0.696274,0.230560
13,0.084678,0.181962
14,0.823097,0.456293


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,170.006645,0.384855
6,75.351000,0.221204
7,476.557653,0.216625
8,677.298267,0.976453
9,570.495055,-0.198326
10,628.371899,0.429112
11,51.747836,0.958435
12,696.273928,-0.538880
13,84.678198,-0.636077
14,823.096922,-0.087414


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
